In [ ]:
from pathlib import Path
import tifffile
import napari
import os
import numpy as np
import pandas as pd
from skimage.measure import regionprops_table
from utils import list_images, read_image

In [33]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/test_data")

# Construct ROI and nuclei predictions paths from directory_path above
roi_path = directory_path / "ROIs"
nuclei_preds_path =  directory_path / "nuclei_preds"

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['raw_data\\test_data\\HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 'raw_data\\test_data\\HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi']

In [34]:
# Explore each image to analyze (0 defines the first image in the directory)
image = images[0]

# Remember to input the same slicing factor you used when generating the nuclei predictions
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Generate maximum intensity projection and extract filename
img_mip, filename = read_image(image, slicing_factor)

# Extract filename with extension
file_and_ext = Path(image).stem

# Show image in Napari
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_mip)

Image displayed: HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1
Original Array shape: (4, 14, 3803, 2891)
MIP Array shape: (4, 3803, 2891)


<Image layer 'img_mip' at 0x29b043e4880>

In [35]:
# List of subfolder names
roi_names = [folder.name for folder in nuclei_preds_path.iterdir() if folder.is_dir()]
print(f"The following regions of interest will be analyzed: {roi_names}")

The following regions of interest will be analyzed: ['CA', 'DG', 'full_image']


In [37]:
# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, cellular_location),(..., ..., ...)]
# Remember in Python one starts counting from 0, so your first channel will be 0

markers = [("ki_67", 0, "nucleus"), ("neun", 1, "nucleus"), ("calbindin", 2, "cytoplasm")]

In [ ]:
for tuple in markers:

    channel_name = tuple[0]
    ch_nr = tuple[1]
    location = tuple[2]



ki_67 0 nucleus
neun 1 nucleus
calbindin 2 cytoplasm


In [50]:
for roi_name in roi_names:

    # Initialize an empty list to hold the extracted dataframes
    props_list = []

    # Read the nuclei predictions per ROI
    nuclei_labels = tifffile.imread(nuclei_preds_path / roi_name / f"{file_and_ext}.tiff")

    # Read the user defined ROIs, in case of full image analysis generate a label covering the entire image
    try:
        user_roi = tifffile.imread(roi_path / roi_name / f"{file_and_ext}.tiff")
    except FileNotFoundError:
        # Extract the xy dimensions of the input image (from nuclei_labels)
        img_shape = nuclei_labels.shape
        img_xy_dims = img_shape[-2:]

        # Create a label covering the entire image
        user_roi = np.ones(img_xy_dims).astype(np.uint8)

    # Add the predicted nuclei as labels into Napari
    viewer.add_labels(nuclei_labels, name=f"{roi_name}_nuclei")

    # Add the predicted nuclei as labels into Napari
    viewer.add_labels(user_roi, name=f"{roi_name}_ROI", opacity=0.4)

    # Loop through each channel and extract the average intensity within either nuclei or cytoplasmic regions
    for tuple in markers:

        channel_name = tuple[0]
        ch_nr = tuple[1]
        location = tuple[2]

        # Extract intensity information from each marker channel
        props = regionprops_table(label_image=nuclei_labels,
                            intensity_image=img_mip[ch_nr],
                            properties=["label", "intensity_mean"])
        
        # Convert to dataframe
        props_df = pd.DataFrame(props)

        # Rename intensity_mean column to indicate the specific image
        props_df.rename(columns={"intensity_mean": f"{channel_name}_intensity_mean"}, inplace=True)

        # Append each props_df to props_list
        props_list.append(props_df)

        props_df


In [51]:
len(props_list)

3